## Install Required Libraries

To load the scraped data from the wikipedia page saved as university_clean.csv into the PostgreSQL database, we will use Python along with the pandas, SQLAlchemy and psycopg2 libraries. psycopg2 library, is the most popular PostgreSQL adapter for Python.
Using pip, Run the following command in the virtual environment:

pip install psycopg2-binary




## Import the necessary libraries

In [1]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, BigInteger, UniqueConstraint
from sqlalchemy.dialects.postgresql import insert



## Connect to PostgreSQL 

#### Connection Parameters:

dbname: The name of your PostgreSQL database.

user: The PostgreSQL username.

password: The PostgreSQL password.

host: The hostname or IP address where your PostgreSQL 
server is running (use localhost if it's running on your local machine).

port: The port number on which your PostgreSQL server is listening (default is 5432).

In [2]:


# Define your database connection parameters
db_params = {
    'dbname': 'amdaridb',
    'user': 'amdariuser',
    'password': 'amdariuserpassword',
    'host': 'localhost',  # Use 'localhost' when accessing from the host machine
    'port': '5432'        # Default PostgreSQL port
}



## SQLAlchemy Engine:

create_engine(connection_string): This function creates a new SQLAlchemy engine instance.

In [3]:
# Create the connection string
connection_string = f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

## Metadata:

MetaData(): Holds a collection of Table objects and their associated schema constructs.

In [4]:
# Define the metadata
metadata = MetaData()


## Table Definition:

Define the table structure using Table() and Column(), specifying the data types and constraints.

In [5]:
df= pd.read_csv('universities_clean.csv')

In [6]:
df.shape

(69, 6)

In [7]:
df.columns

Index(['Country', 'University', 'Founded', 'Type', 'Enrollment', 'Link'], dtype='object')

In [8]:
# Rename DataFrame columns to lowercase
df.rename(columns=str.lower, inplace=True)

In [9]:
df.columns

Index(['country', 'university', 'founded', 'type', 'enrollment', 'link'], dtype='object')

In [10]:
# Define the university table
metadata = MetaData()
university = Table('university', metadata,
    Column('id', BigInteger, primary_key=True, autoincrement=True),
    Column('country', String(255), nullable=False),
    Column('university', String(255), nullable=False),
    Column('founded', Integer, nullable=False),
    Column('type', String(255), nullable=False),
    Column('enrollment', BigInteger, nullable=False),
    Column('link', String(255), nullable=False),
    UniqueConstraint('country', 'university', name='unique_combination'),
    extend_existing=True  # This allows redefining the table
)


## Create Table:

metadata.create_all(engine): This function creates the table in the database if it does not already exist.

In [11]:
# Create the table in the database
metadata.create_all(engine)

print("Table 'university' created successfully.")

Table 'university' created successfully.


In [12]:
# Check if the university table exists
with engine.connect() as connection:
    try:
        result = connection.execute(text("SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = 'university');"))
        exists = result.scalar()
        if exists:
            print("The table 'university' still exists.")
        else:
            print("The table 'university' has been successfully deleted.")
    except Exception as e:
        print(f"Error checking table existence: {e}")


The table 'university' still exists.


In [13]:
# Check the data in the PostgreSQL database
query = "SELECT * FROM university"
df_check = pd.read_sql(query, engine)


In [14]:
df_check.columns

Index(['id', 'country', 'university', 'founded', 'type', 'enrollment', 'link'], dtype='object')

In [15]:
# Display the first few rows of the DataFrame
print(df_check.head())

Empty DataFrame
Columns: [id, country, university, founded, type, enrollment, link]
Index: []


## Insert the dataframe into the university table
Insert the data into the university table using on_conflict_do_update to handle duplicates.

In [16]:
# Check DataFrame columns
print("DataFrame columns:", df.columns)

DataFrame columns: Index(['country', 'university', 'founded', 'type', 'enrollment', 'link'], dtype='object')


In [21]:
# Insert the DataFrame into the university table
with engine.begin() as connection:
    for index, row in df.iterrows():
        insert_statement = text("""
            INSERT INTO university (country, university, founded, type, enrollment, link)
            VALUES (:country, :university, :founded, :type, :enrollment, :link)
            ON CONFLICT (country, university)
            DO UPDATE SET
                founded = EXCLUDED.founded,
                type = EXCLUDED.type,
                enrollment = EXCLUDED.enrollment,
                link = EXCLUDED.link;
        """)
        params = {
            'country': row['country'],
            'university': row['university'],
            'founded': row['founded'],
            'type': row['type'],
            'enrollment': row['enrollment'],
            'link': row['link']
        }
        print(f"Executing: {insert_statement} with params: {params}")
        connection.execute(insert_statement, params)

print("Data loaded successfully.")


Executing: 
            INSERT INTO university (country, university, founded, type, enrollment, link)
            VALUES (:country, :university, :founded, :type, :enrollment, :link)
            ON CONFLICT (country, university)
            DO UPDATE SET
                founded = EXCLUDED.founded,
                type = EXCLUDED.type,
                enrollment = EXCLUDED.enrollment,
                link = EXCLUDED.link;
         with params: {'country': 'Albania', 'university': 'University of Tirana', 'founded': 1957, 'type': 'Public', 'enrollment': 35000, 'link': 'https://en.wikipedia.org/wiki/University_of_Tirana'}
Executing: 
            INSERT INTO university (country, university, founded, type, enrollment, link)
            VALUES (:country, :university, :founded, :type, :enrollment, :link)
            ON CONFLICT (country, university)
            DO UPDATE SET
                founded = EXCLUDED.founded,
                type = EXCLUDED.type,
                enrollment = EXCLUDED.e

In [20]:
# Insert the DataFrame into the university table
with engine.connect() as connection:
    for index, row in df.iterrows():
        insert_statement = text("""
            INSERT INTO university (country, university, founded, type, enrollment, link)
            VALUES (:country, :university, :founded, :type, :enrollment, :link)
            ON CONFLICT (country, university)
            DO UPDATE SET
                founded = EXCLUDED.founded,
                type = EXCLUDED.type,
                enrollment = EXCLUDED.enrollment,
                link = EXCLUDED.link;
        """)
        params = {
            'country': row['country'],
            'university': row['university'],
            'founded': row['founded'],
            'type': row['type'],
            'enrollment': row['enrollment'],
            'link': row['link']
        }
        print(f"Executing: {insert_statement} with params: {params}")
        connection.execute(insert_statement, params)

print("Data loaded successfully.")

Executing: 
            INSERT INTO university (country, university, founded, type, enrollment, link)
            VALUES (:country, :university, :founded, :type, :enrollment, :link)
            ON CONFLICT (country, university)
            DO UPDATE SET
                founded = EXCLUDED.founded,
                type = EXCLUDED.type,
                enrollment = EXCLUDED.enrollment,
                link = EXCLUDED.link;
         with params: {'country': 'Albania', 'university': 'University of Tirana', 'founded': 1957, 'type': 'Public', 'enrollment': 35000, 'link': 'https://en.wikipedia.org/wiki/University_of_Tirana'}
Executing: 
            INSERT INTO university (country, university, founded, type, enrollment, link)
            VALUES (:country, :university, :founded, :type, :enrollment, :link)
            ON CONFLICT (country, university)
            DO UPDATE SET
                founded = EXCLUDED.founded,
                type = EXCLUDED.type,
                enrollment = EXCLUDED.e

In [18]:
# Insert the DataFrame into the university table
with engine.connect() as connection:
    for index, row in df.iterrows():
        insert_statement = text("""
            INSERT INTO university (country, university, founded, type, enrollment, link)
            VALUES (:country, :university, :founded, :type, :enrollment, :link)
            ON CONFLICT (country, university)
            DO UPDATE SET
                founded = EXCLUDED.founded,
                type = EXCLUDED.type,
                enrollment = EXCLUDED.enrollment,
                link = EXCLUDED.link;
        """)
        connection.execute(insert_statement, {
            'country': row['country'],
            'university': row['university'],
            'founded': row['founded'],
            'type': row['type'],
            'enrollment': row['enrollment'],
            'link': row['link']
        })

print("Data loaded successfully.")

Data loaded successfully.


## Verify the Data

In [22]:
# Check the data in the PostgreSQL database
query = "SELECT * FROM university"
df_check = pd.read_sql(query, engine)
print(df_check.columns)
print(df_check)

Index(['id', 'country', 'university', 'founded', 'type', 'enrollment', 'link'], dtype='object')
     id         country                        university  founded    type  \
0   139         Albania              University of Tirana     1957  Public   
1   140         Algeria            Constantine University     1978  Public   
2   141          Angola         Agostinho Neto University     1962  Public   
3   142       Argentina        University of Buenos Aires     1821  Public   
4   143       Australia                 Monash University     1958  Public   
..  ...             ...                               ...      ...     ...   
64  203          Turkey                Anadolu University     1958  Public   
65  204  United Kingdom                   Open University     1969  Public   
66  205   United States              Texas A&M University     1876  Public   
67  206         Uruguay        University of the Republic     1949  Public   
68  207       Venezuela  Universidad Central d

In [23]:
df_check.head()

,id,country,university,founded,type,enrollment,link
0,139,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,140,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,141,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,142,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,143,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University


In [24]:
df_check.shape

(69, 7)

In [ ]:
# Check for any missing data
print("Missing values:\n", df_check.isnull().sum())

Missing values:
 id            0
country       0
name          0
founded       0
type          0
enrollment    0
link          0
dtype: int64


In [ ]:
# Check the types of data in each column
print("Data types:\n", df_check.dtypes)

Data types:
 id             int64
country       object
name          object
founded        int64
type          object
enrollment     int64
link          object
dtype: object
